# **Libraries**

In [ ]:
!pip install timm

In [ ]:
!pip install pytorch_tabnet

In [ ]:
!pip install evaluate

In [ ]:
!pip install warnings

In [ ]:
# Data Processing n' Visualization
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# Compute
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# Data
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

# Random
import os
import random as rand
import timm

In [ ]:
torch.cuda.empty_cache()

In [ ]:
def set_seed(seed):
  rand.seed(seed)
  np.random.seed(seed)
  torch.cuda.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

seed = 123
set_seed(123)

--------------------
# **Data Sample**

In [ ]:
data_dir = '/kaggle/input/wharton-basketball-dataset/games_2022.xlsx'

In [ ]:
df = pd.read_excel(data_dir)
print(f"Dataset Type: {type(df)}")

In [ ]:
df_ts = df

----------------------------
# **Data Preprocessing**

In [ ]:
df_ts = df_ts.drop(columns = ['OT_length_min_tot', 'attendance', 'tz_dif_H_E', 'opponent_team_score', 
                        'team_score', 'home_away', 'notD1_incomplete', 'largest_lead'])
df_ts = df_ts.dropna()
df_ts['home_away_NS'] = df_ts['home_away_NS'].replace({
    1: 1, -1: 0, 0: 2
})

from sklearn.preprocessing import MinMaxScaler

# List of columns to normalize
stats_to_normalize = ['FGA_2', 'FGM_2', 'FGA_3', 'FGM_3', 
                      'FTA', 'FTM', 'AST', 'BLK', 'STL', 'TOV', 
                      'TOV_team', 'DREB', 'OREB', 'F_tech', 'F_personal', 
                      'rest_days', 'prev_game_dist', 'travel_dist']

# Initialize MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))

# Apply MinMaxScaler only to the selected stats
df_ts[stats_to_normalize] = scaler.fit_transform(df_ts[stats_to_normalize])

print(df_ts.head())  # Check normalized values

In [ ]:
Data = {}
X = []
y = []
start_col = 'FGA_2'
end_col = 'F_personal'

start_col_nx = 'rest_days'
end_col_nx = 'travel_dist'
# Loop through all Teams in Dataset
for i ,team in enumerate(df_ts['team'].unique()):
    team_data = df_ts[df_ts['team'] == team]
    for idx in range(len(team_data) - 1):
        # Features = past game (FGA_2 to F_personal) + current game stats (rest_days to travel_dist)
        past_game = team_data.iloc[idx]
        next_game = team_data.iloc[idx + 1]
        
        past_stats = past_game.loc[start_col : end_col].values
        next_stats = next_game.loc[start_col_nx:end_col_nx].values

        # We predict next game FGA_2 to F_personal
        label = next_game.loc[start_col:end_col].values

        if idx == 0 and i == 0:
            combined_stats_x = np.concatenate((past_stats, next_stats))
            print(combined_stats_x)
            print(f"Type of combined_stats: {type(combined_stats_x)}")
            print(f"First Index: {combined_stats_x[0]}")
            print("-"*59)
            print(f"Label: {label}")
            print(f"Type of Label: {type(label)}")
            print(f"First Index: {label[0]}")

        combined_stats = np.concatenate((past_stats, next_stats))
        X.append(combined_stats)
        y.append(label)

In [ ]:
print(f"First Features: {X[0]}")
print(f"Number of Features: {len(X[0])}")

print("-"*59)
print(f"First Labels: {y[0]}")
print(f"Number of Features to Predict: {len(y[0])}")

print(f"Type: {type(X), type(y)}")

In [ ]:
X = np.array(X, dtype=np.float32)
y = np.array(y, dtype=np.float32)

# **Dataset**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.33, random_state=42)

print(X_train.shape)  # Should be (num_samples, num_features) → (N, 19)
print(y_train.shape)  # Should be (num_samples, num_outputs) → (N, 15)
print(f"Type of X_train and X_test: {type(X_train)} | {type(X_test)}")
print(X_train.dtype, y_train.dtype)
print(X_val.dtype, y_val.dtype)
print(X_test.dtype, y_test.dtype)

In [ ]:
import torch
from torch.utils.data import Dataset

class bkb_dataset(Dataset):
    def __init__(self, data, label):
        self.data = data
        self.label = label

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        feature = torch.tensor(self.data[idx], dtype=torch.float32)  # Use float32
        label = torch.tensor(self.label[idx], dtype=torch.float32)  # Use float32

        return {"input_ids": feature, "labels": label}

In [ ]:
train_set = bkb_dataset(
    X_train,
    y_train,
)

val_set = bkb_dataset(
    X_val,
    y_val,
)

test_set = bkb_dataset(
    X_test,
    y_test,
)
print(f"Length of train_set: {len(train_set)}")
print(f"Length of test_set: {len(test_set)}")
print(f"Length of val_set: {len(val_set)}")

In [ ]:
train_batch = 256
test_batch = 32

train_loader = DataLoader(
    train_set,
    batch_size = train_batch,
    shuffle = True
)

val_loader = DataLoader(
    val_set,
    batch_size = test_batch,
    shuffle = False
)

test_loader = DataLoader(
    test_set,
    batch_size = test_batch,
    shuffle = False
)
print(f"Length train_loader: {len(train_loader)}")
print(f"Length test_loader: {len(test_loader)}")
print(f"Length val_loader: {len(val_loader)}")

# **Model**

In [ ]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
HUGGINGFACE_TOKEN = user_secrets.get_secret("HF_TOKEN")

# Login to Hugging Face
login(HUGGINGFACE_TOKEN)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import torch.nn.functional as F

class FeatureGrouping(nn.Module):
    def __init__(self, num_features=19, max_groups=3, embed_dim=4, output_dim=15, resnet_model="resnet50", finetune=False):
        super().__init__()
        self.num_features = num_features
        self.max_groups = max_groups
        self.embed_dim = embed_dim
        self.output_dim = output_dim

        # Embedding for home_away (3 categories: Home, Away, Neutral)
        self.home_away_embed = nn.Embedding(3, embed_dim)

        # Adjust feature count after embedding replacement
        self.adjusted_num_features = num_features - 1 + embed_dim  # 22 total features

        # Calculate the maximum features per group to ensure consistent dimensions
        self.max_features_per_group = self.adjusted_num_features

        # Learnable logits for feature assignment
        self.assignment_logits = nn.Parameter(torch.randn(self.adjusted_num_features, max_groups))

        # Self-Attention layers for each possible number of groups
        self.attention_layers = nn.ModuleDict({
            f"attn_{g}": nn.MultiheadAttention(
                embed_dim=self.max_features_per_group,
                num_heads=1,
                batch_first=True
            )
            for g in range(1, max_groups + 1)
        })

        # Reduce channels before ResNet
        self.channel_reducer = nn.Conv2d(in_channels=max_groups, out_channels=3, kernel_size=1)

        # Pretrained ResNet model
        self.resnet = timm.create_model(resnet_model, pretrained=True)
        in_features = self.resnet.get_classifier().in_features
        self.resnet.reset_classifier(0)

        # Final regression head
        self.fc = nn.Linear(in_features, output_dim)

        # Set ResNet layers to trainable or frozen based on finetune flag
        if finetune:
            for param in self.resnet.parameters():
                param.requires_grad = True
        else:
            for param in self.resnet.parameters():
                param.requires_grad = False

    def forward(self, x):
        batch_size = x.shape[0]

        # Extract home_away index and convert to embeddings
        home_away_idx = x[:, 17].long().clamp(0, 2)
        home_away_embed = self.home_away_embed(home_away_idx)
        x = torch.cat([x[:, :17], home_away_embed, x[:, 18:]], dim=1)

        # Hard feature assignment
        assignment_hard = torch.argmax(self.assignment_logits, dim=1)

        all_group_outputs = []

        # Process different group configurations
        for num_groups in range(1, self.max_groups + 1):
            # Split features into groups
            groups = []
            features_per_group = self.adjusted_num_features // num_groups
            
            for g in range(num_groups):
                start_idx = g * features_per_group
                end_idx = min(start_idx + features_per_group, self.adjusted_num_features)
                group_features = x[:, start_idx:end_idx]
                
                # Pad to match max_features_per_group
                if group_features.shape[1] < self.max_features_per_group:
                    pad_size = self.max_features_per_group - group_features.shape[1]
                    padding = torch.zeros(batch_size, pad_size, device=x.device)
                    group_features = torch.cat([group_features, padding], dim=1)
                
                groups.append(group_features)

            # Process each group with attention
            processed_groups = []
            for g in range(num_groups):
                group_features = groups[g].unsqueeze(1)
                attn_output, _ = self.attention_layers[f"attn_{num_groups}"](
                    group_features, group_features, group_features)
                processed_groups.append(attn_output)

            # Combine processed groups
            group_output = torch.cat(processed_groups, dim=1)
            
            # Pad to match max_groups if necessary
            if num_groups < self.max_groups:
                padding = torch.zeros(
                    batch_size,
                    self.max_groups - num_groups,
                    self.max_features_per_group,
                    device=x.device
                )
                group_output = torch.cat([group_output, padding], dim=1)
            
            all_group_outputs.append(group_output)

        # Stack all configurations
        x_final = torch.stack(all_group_outputs, dim=1)  # [B, max_groups, max_groups, Features]
        
        # Reshape for channel reducer
        x_final = x_final.mean(dim=1)  # [B, max_groups, Features]
        x_final = x_final.permute(0, 2, 1)  # [B, Features, max_groups]
        x_final = x_final.mean(dim=1).unsqueeze(-1).unsqueeze(-1)  # [B, max_groups, 1, 1]
        
        # Apply channel reduction
        x_final = self.channel_reducer(x_final)  # [B, 3, 1, 1]
        
        # Prepare for ResNet
        x_final = x_final.expand(-1, -1, 224, 224)  # [B, 3, 224, 224]

        # Process through ResNet and final layer
        x_final = self.resnet(x_final)
        output = self.fc(x_final)

        return output

# Example usage
batch_size = 1
x = torch.randn(batch_size, 19)  # Example input features
model = FeatureGrouping(resnet_model="resnet50", finetune=False)  # Finetune ResNet
output = model(x)
print(output.shape)  # Expected: (B, 15)
print(output)

In [ ]:
from transformers import PreTrainedModel, PretrainedConfig

class FeatureGroupingConfig(PretrainedConfig):
    model_type = "feature_grouping"

    def __init__(self, num_features=19, output_dim=15, **kwargs):
        super().__init__(**kwargs)
        self.num_features = num_features
        self.output_dim = output_dim

class FeatureGroupingModel(PreTrainedModel):
    config_class = FeatureGroupingConfig

    def __init__(self, config):
        super().__init__(config)
        self.model = FeatureGrouping(
            num_features=config.num_features, 
            output_dim=config.output_dim
        )

    def forward(self, input_ids, labels=None):
        output = self.model(input_ids)

        loss = None
        if labels is not None:
            loss = F.mse_loss(output, labels)  # Mean Squared Error for regression

        return {"loss": loss, "logits": output} if loss is not None else {"logits": output}

In [ ]:
import evaluate

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.array(predictions)
    labels = np.array(labels)

    mse_per_feature = ((predictions - labels) ** 2).mean(axis=0)  # (15,)
    mse_mean = mse_per_feature.mean()  # Overall MSE

    print(f"\n📢 Epoch MSE: {mse_mean:.4f}")  # Print MSE for each epoch

    return {"mse_mean": mse_mean}

In [ ]:
from transformers import TrainerCallback

class LossLoggerCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs and "loss" in logs:
            print(f"📢 Epoch {state.epoch:.0f}: Train Loss = {logs['loss']:.4f}")

In [ ]:
import warnings
from transformers import Trainer, TrainingArguments

# Ignore all warnings
warnings.filterwarnings("ignore")

training_args = TrainingArguments(
    output_dir="./WhartonDS_RegressionModel",
    learning_rate=1e-5,
    eval_strategy="epoch",  # Evaluate at each epoch
    save_strategy="epoch",
    logging_strategy="epoch",  # Log every epoch
    logging_dir="./logs",
    logging_steps=1,  # Log at every step
    per_device_train_batch_size=128,
    per_device_eval_batch_size=32,
    num_train_epochs=60,
    load_best_model_at_end = True,
    push_to_hub=True,
    optim='adamw_torch',
    report_to="none"
)

# Initialize Model
model = FeatureGroupingModel(FeatureGroupingConfig())

trainer = Trainer(
    model=model.to(device),
    args=training_args,
    train_dataset=train_set,
    eval_dataset=val_set,
    #callbacks=[LossLoggerCallback()]  # Add the callback here
)

# Train the Model
trainer.train()

# Save Model
trainer.save_model("./WhartonDS_RegressionModel")

# Save Model to Hugging Face Hub
trainer.push_to_hub("KanWasTaken/RegressionModel_4DataGen_Wharton")

# **Test**

In [ ]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(X, y, test_size=0.3, random_state=42)
X_val1, X_test1, y_val1, y_test1 = train_test_split(X_test1, y_test1, test_size=0.33, random_state=42)

print(X_train1.shape)  # Should be (num_samples, num_features) → (N, 19)
print(y_train1.shape)  # Should be (num_samples, num_outputs) → (N, 15)

In [ ]:
from sklearn.multioutput import MultiOutputRegressor
from xgboost import XGBRegressor

# Wrap XGBRegressor to handle multi-output regression
model = MultiOutputRegressor(XGBRegressor(
    objective='reg:squarederror',  
    n_estimators=100,  
    learning_rate=0.1  
))

# Train model
model.fit(X_train1, y_train1)  

y_pred = model.predict(X_test1)
print(y_pred.shape)  # Should be (N_test, 15)

from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(y_test1, y_pred, multioutput='raw_values')  # RMSE for each label
print("RMSE per output:", rmse)
print("Average RMSE:", rmse.mean())  # Average RMSE across all 15 labels

In [ ]:
X_pred_home = model.predict(np.array([X_train1[0]]))  # Ensures a 2D shape (1, 19)
X_pred_away = model.predict(np.array([X_train1[1]]))
print(X_pred_home)
print(X_pred_away)

In [ ]:
from pytorch_tabnet.tab_model import TabNetRegressor

X_train1 = X_train1.astype(np.float32)
y_train1 = y_train1.astype(np.float32)
X_val1 = X_val1.astype(np.float32)
y_val1 = y_val1.astype(np.float32)

X_test1 = X_test1.astype(np.float32)
y_test1 = y_test1.astype(np.float32)

if y_test1.ndim == 1:
    y_test1 = y_test1.reshape(-1, 1)

if y_train1.ndim == 1:
    y_train1 = y_train1.reshape(-1, 1)
if y_val1.ndim == 1:
    y_val1 = y_val1.reshape(-1, 1)
    
model = TabNetRegressor()
model.fit(X_train1, y_train1, eval_set=[(X_val1, y_val1)])
preds = model.predict(X_test1)